In [ ]:
# Importamos la clase Model de pyscipopt
from pyscipopt import Model
# Importamos numpy
import numpy as np

In [10]:
model = Model("Alquiler")

In [11]:
# Variables exogenas

utilidad = np.array([
    [12, 22, 38, 40],
    [0, 13, 20, 29],
    [0, 0, 10, 19],
    [0, 0 ,0, 12]
    ])

In [12]:
# Variables de decision

x = np.empty((4,4), dtype=object)
for i in range(4):
    for j in range(4):
        x[i,j] = model.addVar(vtype="B", name=f"x_{i + 1}->{j + 2}")

In [ ]:
# Restricciones

for i in range(4):
    model.addCons(np.sum(x[i,:]) <= 1, name="no se repite inicio de contrato")
    model.addCons(np.sum(x[:,i]) <= 1, name="no se repite fin de contrato")

for i in range(4):
    for j in range(4):
        model.addCons(x[i,j] + np.sum(x[i:j,:4]) <= 1, name=f"no se pisan contratos {i} {j}")

In [14]:
model.setObjective(np.sum(utilidad * x), "maximize")

In [15]:
# Optimizar
model.redirectOutput()
model.optimize()

sol = model.getBestSol()

# Imprimimos la solucion
for v in model.getVars():
    print(f"{v.name}: {model.getSolVal(sol, v)}")

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       6 del vars, 12 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 11 clqs
(round 2, exhaustive) 6 del vars, 20 del conss, 0 add conss, 0 chg bounds, 7 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 11 clqs
(round 3, exhaustive) 6 del vars, 20 del conss, 0 add conss, 0 chg bounds, 7 chg sides, 0 chg coeffs, 4 upgd conss, 0 impls, 11 clqs
(round 4, fast)       11 del vars, 21 del conss, 0 add conss, 0 chg bounds, 7 chg sides, 0 chg coeffs, 4 upgd conss, 0 impls, 2 clqs
(round 5, fast)       13 del vars, 23 del conss, 0 add conss, 0 chg bounds, 7 chg sides, 0 chg coeffs, 4 upgd conss, 0 impls, 1 clqs
presolving (6 rounds: 6 fast, 3 medium, 3 exhaustive):
 16 deleted vars, 24 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 7 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transforme

In [16]:
model.writeSol(sol, 'soluciones/ejercicio_6.sol')